#### Протоколы, duck typing, гусиная типизация и прочая

**Динамическая типизация**
В языках со статической типизацией тип переменной определяется во время объявления и гарантировать, что она окажется заявленного типа - можно: если существует хоть один вариант того, что тип окажется не такой, как заявлен - будет или ошибка компиляции, исполняемый файл просто не создастся и запускать будет нечего, или программа просто не запустится. 

В python динамическая типизация: тип переменной определяется во время выполнения программы и заранее гарантировать, что она окажется нужного типа - нельзя. Может так случиться, что тип окажется не такой, как заявлен - будет либо ошибка выполнения, либо молча и неявно сделается не то, что ожидалось, потому что тип - другой. Нужны какие-то гарантии, что всегда переменные будут нужного ожидаемого типа. 

```python
class Rectangle:
    def __init__(self, width: int, height: int):
        self._width = width
        self._height = height

    def get_area(self):
        return self._width * self._height

if __name__ == "__main__":
    r = Rectangle(10, "tuesday")
    print(r.get_area())
```

Ну умножили вторник на ширину, что-то даже получилось, кто и что будет дальше делать с этим результатом и что получится - как повезет.

Как обеспечить эти гарантии

`isinstance` в функциях, методах класса и везде проверяет является ли тип переменной тем, что ожидается, с учетом наследования. Приведет к тому, что объем кода вырастет в два раза и будет читать крайне тяжело. Кроме того, это приводит к увеличению времени работы программы.


In [3]:
import abc

class Flyable(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass

class Junkie(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass

class Bird(Flyable):
    def fly(self):
        print("I'm flying")

class Hippie(Junkie):
    def fly(self):
        print("I'm flying high")

class FlyableManager:
    def __init__(self):
        self.flyables: list[Flyable] = [] # список объектов, которые могут летать

    def add_flyable(self, flyable: Flyable):
        # if not isinstance(flyable, Flyable):
        #     # здесь мы проверяем, что переданный объект является Flyable
        #     raise TypeError("FlyableManager can only add Flyable objects")
        self.flyables.append(flyable)
    
    def fly(self):
        for flyable in self.flyables:
            flyable.fly()

manager = FlyableManager()
manager.add_flyable(Bird())
# manager.add_flyable("tuesday")
# несмотря на то, что Hippie точно так же летает, мы не можем добавить его в менеджер
manager.add_flyable(Hippie())
manager.fly()

<cell>23: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]
<cell>39: error: Argument 1 to "add_flyable" of "FlyableManager" has incompatible type "Hippie"; expected "Flyable"  [arg-type]


I'm flying
I'm flying high


##### Протоколы и duck typing
Декларируем набор методов - протокол. Наследоваться не надо ни от чего: объекты тех классов, у которых объявлены нужные методы соответствуют протоколу. Если что-то ходит, как утка, и квакает, как утка, - значит, может быть использовано в качестве утки по части ходьбы и кваканья.

In [6]:
%load_ext nb_mypy
import random
from typing import  Protocol

class Flyable(Protocol):
    def fly(self):
        pass

class Bird():
    def fly(self):
        print("I'm flying")

class Hippie():
    def fly(self):
        print("I'm flying high")

class ParametrizedFlyer():
    def fly(self, speed: int):
        print(f"I'm flying at {speed} knots")

class FlyableManager:
    def __init__(self):
        self.flyables = []

    def add_flyable(self, flyable: Flyable):
        # Из коробки не заработает, потому что протоколы для isinstance нужно дополнительно размечать
        # if not isinstance(flyable, Flyable):
        #     # здесь мы проверяем, что переданный объект является Flyable
        #     raise TypeError("FlyableManager can only add Flyable objects")
        self.flyables.append(flyable)
    
    def fly(self):
        for flyable in self.flyables:
            flyable.fly()

manager = FlyableManager()
manager.add_flyable(Bird())
# несмотря на то, что Hippie никак не связан явно с протоколом Flyable - пожалуйста
manager.add_flyable(Hippie())

manager.fly()
# а вот это уже не работает
# manager.add_flyable(ParametrizedFlyer())
# это тоже не работает
if random.random() > 0.999:
    manager.add_flyable(1)


<cell>46: error: Argument 1 to "add_flyable" of "FlyableManager" has incompatible type "int"; expected "Flyable"  [arg-type]


The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy
I'm flying
I'm flying high


#### Задачи - правильные скобочные последовательности (2 октября)

Строка, состоящая из символов `(` и `)`, называется правильной скобочной последовательностью, если:

1. пустая строка является правильной скобочной последовательностью;
2. если `A` и `B` являются правильными скобочными последовательностями, то `AB` также является правильной скобочной последовательностью.
3. если `B` является правильной скобочной последовательностью, то `(B)` также является правильной скобочной последовательностью.

За O(N) по времени и O(1) по памяти определить, является ли некоторая строка правильной скобочной последовательностью.

Обобщение (2 балла): пусть у нас есть несколько видов скобок. Для валидности считаем, что закрываться должна последняя открытая скобка.
То есть `[]()` - валидная скобочная последовательность, а `[(])` - нет.

За O(N) по времени и O(N) по памяти определить, является ли некоторая строка правильной скобочной последовательностью c несколькими типами скобок  